In [2]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

![CRISP_DM](CRISP_DM.png )

# Groupby apply on large (relational) data set

In [3]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,"Korea, South",1.0
2,2020-01-22,no,Kosovo,0.0
3,2020-01-22,no,Kuwait,0.0
4,2020-01-22,no,Kyrgyzstan,0.0


In [4]:
temp = pd.read_csv('../data/external/202004_covid19countryinfo_clean.csv',sep=';',parse_dates=[0])

In [5]:
temp

,country,pop,density,medianage,urbanpop,quarantine,schools,publicplace,hospibed,smokers,...,sexratio,lung,femalelung,malelung,gdp2019,healthexp,healthperpop,fertility,avgtemp,avghumidity
0,Afghanistan,"38,928,346",60.0,18.0,25.0,NaN,NaN,NaN,0.5000,NaN,...,1.03,37.62,36.31,39.33,"18,734",184,4.726633,5.12,6.216667,56.933333
1,Albania,"2,877,797",105.0,36.0,63.0,NaN,NaN,NaN,2.9000,29.40,...,0.98,11.67,7.02,17.04,"15,418",774,268.955733,1.51,13.950000,69.900000
2,Algeria,"43,851,044",18.0,29.0,73.0,NaN,NaN,NaN,1.9000,NaN,...,1.03,8.77,5.03,12.81,"172,781","1,031",23.511413,2.70,19.566667,61.716667
3,Andorra,"77,265",164.0,45.0,88.0,NaN,NaN,NaN,2.5000,32.50,...,1.06,NaN,NaN,NaN,"91,527","5,949",76994.758300,1.40,7.383333,75.233333
4,Antigua and Barbuda,"97,929",223.0,34.0,26.0,NaN,NaN,NaN,3.8000,NaN,...,0.90,11.76,7.67,18.78,"1,688","1,105",11283.685120,2.00,NaN,NaN
5,Argentina,"45,195,774",17.0,32.0,93.0,3/20/2020,NaN,NaN,5.0000,23.95,...,0.98,29.27,20.16,42.59,"445,469","1,390",30.755088,2.26,32.816667,43.616667
6,Armenia,"2,963,243",104.0,35.0,63.0,NaN,NaN,NaN,4.2000,26.90,...,0.94,23.86,16.17,35.99,"13,444",883,297.984337,1.64,7.100000,58.500000
7,Australia,"25,499,884",3.0,38.0,86.0,NaN,NaN,3/23/2020,3.8000,14.90,...,0.99,18.79,15.90,22.16,"1,376,255","4,492",176.157664,1.77,25.783333,64.233333
8,Austria,"9,006,398",109.0,43.0,57.0,3/16/2020,NaN,NaN,7.6000,35.15,...,0.96,17.02,13.02,22.14,"447,718","5,138",570.483339,1.47,9.466667,65.350000
9,Azerbaijan,"10,139,177",123.0,32.0,56.0,NaN,NaN,NaN,4.7000,23.45,...,0.98,20.61,14.07,29.32,"47,171","1,191",117.465155,1.89,10.166667,68.083333


## Test Data

In [4]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-03-20')]

In [5]:
test_data.head()

,date,state,country,confirmed
15737,2020-03-21,no,Germany,22213.0
15777,2020-03-21,no,US,25600.0
16003,2020-03-22,no,Germany,24873.0
16043,2020-03-22,no,US,33280.0
16268,2020-03-23,no,Germany,29056.0


In [6]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-05-22,no,179710.0
US,2020-05-22,no,1600937.0


In [16]:
# %load ../src/features/build_features.py

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)

        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result

        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here

    result=signal.savgol_filter(np.array(filter_in),
                           window, # window size used for filtering
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='confirmed'):
    ''' Rolling Regression to approximate the doubling time'

        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['state','country',filter_on]].groupby(['state','country']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())

    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()





def calc_doubling_rate(df_input,filter_on='confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['state','country']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output


if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

    pd_JH_data=pd.read_csv('data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()

    #test_structure=pd_JH_data[((pd_JH_data['country']=='US')|
    #                  (pd_JH_data['country']=='Germany'))]

    pd_result_larg=calc_filtered_data(pd_JH_data)
    pd_result_larg=calc_doubling_rate(pd_result_larg)
    pd_result_larg=calc_doubling_rate(pd_result_larg,'confirmed_filtered')


    mask=pd_result_larg['confirmed']>100
    pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)
    pd_result_larg.to_csv('data/processed/COVID_final_set.csv',sep=';',index=False)
    print(pd_result_larg[pd_result_larg['country']=='Germany'].tail())


the test slope is: [2.]


FileNotFoundError: [Errno 2] File data/processed/COVID_relational_confirmed.csv does not exist: 'data/processed/COVID_relational_confirmed.csv'

In [17]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-05-22   179710.0
      US      2020-05-22  1600937.0

In [20]:
#test_data.groupby(['state','country']).apply(get_doubling_time_via_regression)

In [21]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [22]:

test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country       
no     Germany  15737           NaN
                16003           NaN
                16268      7.417994
                16535      7.142035
                16801      8.012983
                17067      6.954407
                17333      6.501919
                17599      7.390371
                17865     10.136671
                18131     13.541893
                18396     13.781393
                18663     13.140681
                18929     12.037271
                19194     12.735506
                19462     16.052694
                19727     21.372453
                19993     27.427355
                20259     27.511936
                20524     21.792179
                20791     21.495848
                21057     26.565108
                21323     36.198404
                21588     43.982990
                21855     49.423444
                22121     74.043747
                22387     56.424411
                22652     42.468318
      

In [23]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()


In [24]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR',
                             'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,465,NaN
2,Alberta,Canada,701,NaN
3,Alberta,Canada,966,NaN
4,Alberta,Canada,1232,NaN


In [25]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0.0
1,1,2020-01-22,no,"Korea, South",1.0
2,2,2020-01-22,no,Kosovo,0.0
3,3,2020-01-22,no,Kuwait,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0


In [26]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN


## Filtering the data with groupby apply

In [27]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result

In [28]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [29]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,2020-01-22,no,"Korea, South",1.0,NaN,0.8
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0


## Filtered doubling rate

In [32]:
pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
32447,no,Zimbabwe,31217,29.200000
32448,no,Zimbabwe,31452,30.266667
32449,no,Zimbabwe,31748,29.291667
32450,no,Zimbabwe,31984,33.402299
32451,no,Zimbabwe,32451,33.266667


In [33]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
32447,32447,2020-05-22,no,Barbados,90.0,inf,90.4,225.000000
32448,32448,2020-05-22,no,Belarus,34303.0,35.553188,34301.0,35.783914
32449,32449,2020-05-22,no,Belgium,56511.0,213.041667,56485.4,239.568143
32450,32450,2020-05-22,no,Albania,981.0,114.274510,979.4,112.883721
32451,32451,2020-05-22,no,Zimbabwe,51.0,33.333333,51.4,33.266667


In [34]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)

In [35]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
31164,31164,2020-05-18,no,Germany,176551.0,441.111389,176984.6,269.621713
31430,31430,2020-05-19,no,Germany,177778.0,251.099125,177638.4,271.910483
31695,31695,2020-05-20,no,Germany,178473.0,184.808186,178306.6,268.749168
31963,31963,2020-05-21,no,Germany,179021.0,287.086082,179062.7,250.419013
32228,32228,2020-05-22,no,Germany,179710.0,289.519806,179818.8,236.824097


In [36]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)